## Spark SQL, DataFrame

###  데이터 종류
- 비 구조화된 데이터
    - 형식이 없음
    - 로그 파일, 이미지
- 준 구조화된 데이터
    - 행과 열을 가짐
    - csv, json, xml
- 구조화된 데이터
    - 행과열 + ``데이터 타입(스키마)를 가진다.``
    - 데이터베이스
     <br>
- DataLake : 비구조화된 데이터, 준구조화된 데이터
- Datawarehouse : 비구조화된 데이터, 준구조화된 데이터
- Datamart : 구조화된 데이터

### Spark SQL 목적
- 스파크 프로그래밍 내부에서 관계형 처리를 하기 위함 
- 스키마의 정보를 이용해 ``자동으로 최적화``를 하기 위함
- 외부 데이터 세트를 사용하기 쉽게 하기 위함 
 <br>
- 2개의 벡엔드 컴포넌트가 존재
    - Catalyst - 쿼리 최적화 엔진(파티션 및 셔플링 최적화) - 논리적인 최적화
    - Tungsten - 시리얼라이저(데이터 용량 최적화) - 물리적 제어

### DataFrame
- RDD에 스키마가 적용 된 것 

### SparkSession - Spark SQL에 있다.   

In [1]:
import findspark
findspark.init()

## 1. SparkSession 만들기
 - sparkContext 에 해당하며 , 새로운 스파크 어플리케이션을 만들어 준다.

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()
# spark 네임 바꾸면 안된다.

In [5]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
] # 일반 list

In [6]:
# 정형데이터를 만들 것이다.
movies_schema = ["id","name","company","year","month","day"]

## 2. 데이터 프레임 만들기

In [7]:
# spark가 유추해서 데이터 타입을 결정한다. -> inferSchema=True이면 알아서 schema 정해준다.
df=spark.createDataFrame(data=movies, schema=movies_schema)
df

DataFrame[id: bigint, name: string, company: string, year: bigint, month: bigint, day: bigint]

In [10]:
# 스키마 타입 확인 `dtypes`
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [11]:
# 전체 데이터 프레임 내용 확인
df.show() 

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



## 3. Spark SQL 사용하기
- `createOrReplaceTempView` 함수를 이용해서 DataFrame을 table 형식으로 등록

In [13]:
df.createOrReplaceTempView("movies") # table 이름 => movies

In [17]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [15]:
# 이제부터 쿼리 사용가능
query="""
select name
from movies
"""

In [16]:
# 쿼리 실행
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [18]:
# 영화 이름, 개봉연도 가져오기
query='''
select name,year
from movies
'''
spark.sql(query).show()

+--------+----+
|    name|year|
+--------+----+
|어벤져스|2012|
|  슈퍼맨|2013|
|  배트맨|2008|
|겨울왕국|2014|
|아이언맨|2008|
+--------+----+



In [19]:
# 2010년 이후 개봉한 영화의 모든 정보
query='''
select *
from movies
where year > 2010
'''
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [21]:
# 마블 영화 중에 2010년 이후 개봉한 영화의 모든 정보
# 영화 이름, 개봉연도 가져오기
query='''
select *
from movies
where company ="마블" and year >2010
'''
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
+---+--------+-------+----+-----+---+



In [31]:
# 영화의 이름이 ~맨으로 끝나는 영화의 모든 정보
query='''
select *
from movies
where name like "%맨"
'''
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [42]:
# 개봉월이 4~8월인 영화의 모든 정보
query='''
select *
from movies
where month >=4 and month <= 8
'''
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [49]:
# id가 3번인 영화보다 늦게 개봉한 마블영화의 모든 정보
query='''


select year
from movies
where id=3
'''
spark.sql(query).show()

+----+
|year|
+----+
|2008|
+----+



In [51]:
# id가 3번인 영화보다 늦게 개봉한 마블영화의 모든 정보
query='''

select *
from movies
where company='마블' and year > 
(select year
from movies
where id=3)
'''
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
+---+--------+-------+----+-----+---+



# 개발자가 직접 스키마 정의
- spark 스트리밍 사용 시 무조건 해줘야 한다.

In [52]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [1]:
# 자료형 타입 불러오기
from pyspark.sql.types import StringType, FloatType, IntegerType

# 구조를 만들기 위한 타입 불러오기(필수) -> 스키마를 정상적으로 만들려면 꼭 필요하다
from pyspark.sql.types import StructType, StructField # StructType 은 스키마 정보를 모든 정보들을 모아둔거

``중요한 스키마 수동으로 구성하기``

In [70]:
# StructType : 스키마를 구성하는 모든 정보들을 묶어주는 역할
attendances_schema= StructType([
    # 컬럼의 정보를 구조화 시켜주는 역할(컬럼명, 컬럼 타입, null 허용 여부)
    StructField('id',IntegerType(),True),
    StructField('attendance',FloatType(),True),
    StructField('theater_country', StringType(),True) 
])

In [63]:
att_df=spark.createDataFrame(data=attendances,schema=attendances_schema)
att_df.dtypes

[('id', 'int'), ('attendance', 'float'), ('theater_country', 'string')]

In [64]:
att_df.createOrReplaceTempView("att")

In [66]:
query= '''
select *
from movies
join att on movies.id = att.id
'''
spark.sql(query).show()

+---+--------+-------+----+-----+---+---+-----------+---------------+
| id|    name|company|year|month|day| id| attendance|theater_country|
+---+--------+-------+----+-----+---+---+-----------+---------------+
|  1|어벤져스|   마블|2012|    4| 26|  1|1.3934592E7|             KR|
|  2|  슈퍼맨|     DC|2013|    6| 13|  2|  2182227.0|             KR|
|  3|  배트맨|     DC|2008|    8|  6|  3|  4226242.0|             KR|
|  4|겨울왕국| 디즈니|2014|    1| 16|  4|1.0303058E7|             KR|
|  5|아이언맨|   마블|2008|    4| 30|  5|  4300365.0|             KR|
+---+--------+-------+----+-----+---+---+-----------+---------------+



In [68]:
spark.stop()